<a href="https://colab.research.google.com/github/meyresearch/ANI-Peptides/blob/main/demos/AMBER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Runtime Environment Setup

Before you start, ensure you have been allocated a GPU runtime. Go to `Runtime -> Change runtime type -> GPU`

Allow kernel to restart after running this cell.

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

Ensure good conda/mamba installation

In [ ]:
import condacolab
condacolab.check()

Install Dependencies



*   *OpenMM* - Molecular dynamics simulation
*   *NGLView, PyTraj* - Viewing libraries



In [ ]:
!mamba install -q openmm nglview
!pip install -q pytraj

Test OpenMM

In [ ]:
!python -m openmm.testInstallation

Fetch sample peptide

In [4]:
!wget -q https://github.com/meyresearch/ANI-Peptides/raw/main/demos/aaa.pdb

# AMBER Simulation

In [5]:
# Import libraries

from openmm.app import *
from openmm import *
from openmm.unit import *

In [6]:
# Progress bar for simulations

from IPython.display import ProgressBar

class ProgressBarWriter(ProgressBar):
    def write(self, string):
        if string.startswith("#"):
            self.progress = 0
        else:
            string = string.strip(',\n')
            try:
                self.progress = int(string)
            except ValueError:
                pass

def ProgressBarReporter(steps, interval):
    return StateDataReporter(
        ProgressBarWriter(total = steps), 
        interval, 
        step = True,
        totalSteps = steps,
    )

In [7]:
# Simulation function (to run simulations consistently)

def simulate(
    simulation,
    output_pdb_filename: str = "output.pdb",
    output_state_data_filename: str = "state_data.csv",
    step_size: Quantity = 0.004*femtoseconds,
    steps: int or None = 100000,
    duration: Quantity or None = None,
    pdb_report_interval: int or None = None,
    state_report_interval: int or None = None,
    progressbar_interval: int or None = None,
):
    assert duration or steps

    if duration:
        steps = int(duration / step_size)

    if not pdb_report_interval:
        pdb_report_interval = steps // 10

    if not state_report_interval:
        state_report_interval = steps // 10

    if not progressbar_interval:
        progressbar_interval = steps // 50

    # Create pdb file reporter
    simulation.reporters.append(
        PDBReporter(
            output_pdb_filename,
            pdb_report_interval
        )
    )

    # Create state data file reporter
    simulation.reporters.append(
        StateDataReporter(
            output_state_data_filename, 
            state_report_interval, 
            step=True,
            potentialEnergy=True, 
            temperature=True
        )
    )

    # Create progress bar reporter
    simulation.reporters.append(
        ProgressBarReporter(steps, progressbar_interval)
    )

    # Run simulation through specified time steps
    print("Running simulation...")
    simulation.step(steps)

In [8]:
# Equilibration function - Constant pressure & temp

def equilibrate(
    coords: Topology,
    forcefield: ForceField,
    pressure: Quantity = 1*atmosphere,
    temperature: Quantity = 300*kelvin,
    friction_coeff: Quantity = 1/femtosecond,
    step_size: Quantity = 0.004*femtoseconds,
    **kwargs
):      
    print("Initialising equilibration run...")

    # Create system
    system = forcefield.createSystem(
        coords.topology, 
        nonbondedMethod=PME,
        nonbondedCutoff=1*nanometer,
        constraints=AllBonds,
        hydrogenMass=4*amu,
    )

    # Add a barostat for constant pressure
    system.addForce(
        MonteCarloBarostat(
            pressure,
            temperature
        )
    )

    # Create constant temp integrator
    integrator = LangevinMiddleIntegrator(
        temperature,
        friction_coeff,
        step_size
    )

    # Create simulation and set initial positions
    simulation = Simulation(
        coords.topology,
        system,
        integrator
    )
    simulation.context.setPositions(coords.positions)

    # Local energy minimisation
    print("Local energy minimisation...")
    simulation.minimizeEnergy()

    simulate(
        simulation,
        **kwargs
    )

    return simulation

In [9]:
# Production function - Constant volume & temp

def production(
    coords: Topology,
    forcefield: ForceField,
    temperature: Quantity = 300*kelvin,
    friction_coeff: Quantity = 1/femtosecond,
    step_size: Quantity = 0.004*femtoseconds,
    **kwargs
):      
    print("Initialising production run...")

    # Create system
    system = forcefield.createSystem(
        coords.topology, 
        nonbondedMethod=PME,
        nonbondedCutoff=1*nanometer,
        constraints=AllBonds,
        hydrogenMass=4*amu,
    )

    # Create constant temp integrator
    integrator = LangevinMiddleIntegrator(
        temperature,
        friction_coeff,
        step_size
    )

    # Create simulation and set initial positions
    simulation = Simulation(
        coords.topology,
        system,
        integrator
    )
    simulation.context.setPositions(coords.positions)

    simulate(
        simulation,
        **kwargs
    )

    return simulation

In [ ]:
# Load sample peptide
pdb = PDBFile('aaa.pdb')

# Create AMBER forcefield
forcefield = ForceField(
    'amber14-all.xml',
    'amber14/tip3pfb.xml'
)

# Load pdb into modeller and add solvent
modeller = Modeller(pdb.topology, pdb.positions)
modeller.addSolvent(forcefield, model='tip3p', padding=1*nanometer)

simulation = equilibrate(
    modeller,
    forcefield,
    pressure = 1*atmosphere,
    temperature = 300*kelvin,
    friction_coeff = 1/femtosecond,
    step_size = 0.004*femtoseconds,
    steps = 10000,
    output_pdb_filename = "equilibration_output.pdb",
    output_state_data_filename = "equilibration_state_data.csv"
)

simulation.positions = simulation.context.getState(getPositions=True).getPositions()

production(
    simulation,
    forcefield,
    temperature = 300*kelvin,
    friction_coeff = 1/femtosecond,
    step_size = 0.004*femtoseconds,
    steps = 10000,
    output_pdb_filename = "production_output.pdb",
    output_state_data_filename = "production_state_data.csv"
)

# Viewer

In [ ]:
import pytraj as pt
import nglview as nv
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
traj = pt.load("production_output.pdb")
view = nv.show_pytraj(traj)
view.add_representation("licorice")
view.add_unitcell()
view